### Creating and Persisting an ML Model

In [134]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/student-mat.csv', sep=';')

Summary of the data

In [135]:
df.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861,10.908861,10.713924,10.415190
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096,3.319195,3.761505,4.581443
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,3.000000,0.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000,8.000000,9.000000,8.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000,11.000000,11.000000,11.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000,13.000000,13.000000,14.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000,19.000000,19.000000,20.000000


In [136]:
df.info

<bound method DataFrame.info of     school sex  age address famsize Pstatus  Medu  Fedu      Mjob      Fjob  \
0       GP   F   18       U     GT3       A     4     4   at_home   teacher   
1       GP   F   17       U     GT3       T     1     1   at_home     other   
2       GP   F   15       U     LE3       T     1     1   at_home     other   
3       GP   F   15       U     GT3       T     4     2    health  services   
4       GP   F   16       U     GT3       T     3     3     other     other   
..     ...  ..  ...     ...     ...     ...   ...   ...       ...       ...   
390     MS   M   20       U     LE3       A     2     2  services  services   
391     MS   M   17       U     LE3       T     3     1  services  services   
392     MS   M   21       R     GT3       T     1     1     other     other   
393     MS   M   18       R     LE3       T     3     2  services     other   
394     MS   M   19       U     LE3       T     1     1     other   at_home   

     ... famrel fre

Create a subset of features as an example.

In [137]:
include = ['health', 'absences','age','G1','G2','G3']
df.drop(columns=df.columns.difference(include), inplace=True)  # only using 3 features

In [138]:
df.info

<bound method DataFrame.info of      age  health  absences  G1  G2  G3
0     18       3         6   5   6   6
1     17       3         4   5   5   6
2     15       3        10   7   8  10
3     15       5         2  15  14  15
4     16       5         4   6  10  10
..   ...     ...       ...  ..  ..  ..
390   20       4        11   9   9   9
391   17       2         3  14  16  16
392   21       3         3  10   8   7
393   18       5         0  11  12  10
394   19       5         5   8   9   9

[395 rows x 6 columns]>

The goal is to predict the quality of the student. We will build a predictor based on the final grade (G3).
Becasue we are trying to find quality students. In this model we define a quality student as one who achieves a final grade of 15 or higher. 

In [139]:
df['qual_student'] = np.where(df['G3']>=15, 1, 0)

In [140]:
df.describe()

,age,health,absences,G1,G2,G3,qual_student
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,3.554430,5.708861,10.908861,10.713924,10.415190,0.184810
std,1.276043,1.390303,8.003096,3.319195,3.761505,4.581443,0.388636
min,15.000000,1.000000,0.000000,3.000000,0.000000,0.000000,0.000000
25%,16.000000,3.000000,0.000000,8.000000,9.000000,8.000000,0.000000
50%,17.000000,4.000000,4.000000,11.000000,11.000000,11.000000,0.000000
75%,18.000000,5.000000,8.000000,13.000000,13.000000,14.000000,0.000000
max,22.000000,5.000000,75.000000,19.000000,19.000000,20.000000,1.000000


Drop the G3 score

In [141]:
include = ['health', 'absences','age','G1','G2','qual_student']
df.drop(columns=df.columns.difference(include), inplace=True) 

Import scikit-learn and build a random forest classifer

In [143]:
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split
import sklearn
dependent_variable = 'qual_student'
x = df[df.columns.difference([dependent_variable])]
y = df[dependent_variable]

#splitting the train and test sets
X_train, X_test, y_train,y_test= train_test_split(x, y, test_size=0.5)

clf = rf(n_estimators = 1000)
clf.fit(X_train, y_train)

# Determine the training accuracy of the model
print("Training Accuracy {:.4%}".format(clf.score(X_train, y_train)))

Training Accuracy 100.0000%


In [144]:
# Determine the test accuracy of the model
print("Test Accuracy {:.4%}".format(clf.score(X_test, y_test)))

Test Accuracy 96.9697%


Let's export this model so we can use it in a microservice (flask api)

In [145]:
import joblib
# modify the file path to where you want to save the model
joblib.dump(clf, './dockerfile/apps/model.pkl')

['./dockerfile/apps/model.pkl']

In [152]:
query_df = pd.DataFrame({'G1' : pd.Series(15), 'G2' : pd.Series(15), 'absences' : pd.Series(10), 'age' : pd.Series(1), 'health' : pd.Series(15)})

In [153]:
pred = clf.predict(query_df)

In [154]:
pred

array([1])